In [23]:
# Create EJI Tool Version: Beta (5/14/2021)

# This script downloads JSON data from a Census Bureau API for 5-Year ACS Estimates, performs calculations and data formatting in
# Pandas dataframes, and uses ESRI (arcpy) tools to join the results to Block Group and Tract geographies. Outputs are created in
# file geodatabase and CSV formats. It is intended to automate annual production of NCTCOG's Transit Accessibility Improvement Tool (TAIT).
# It is in large part based on a similar tool to streamline production of the annual Environmental Justice Index (EJI).
# Key parameters  are specified by the user in the tool interface, but additional parameters can be customized below.

#Import some libraries
# import arcpy
import os
import urllib3
import json
import numpy as np
import pandas as pd

call = urllib3.PoolManager()

In [33]:
# List of dictionaries describing the data you're requesting from the Census API at the BLOCK GROUP geography.
# This has to be split into two queries because 50 variables is the upper limit for requests to this API.
# desc_name is a descriptive name of your choosing. This becomes part of the field name. Choose something tidy
# that will result in a recognizable field name later.
# census_name is the variable name as described here: https://api.census.gov/data/2019/acs/acs5/variables.html

bg_desired_col1 =    [{'desc_name': 'Total_Pop', 'census_name': 'B01001_001E'},
                      {'desc_name': 'NotHispLatino_WhiteAlone', 'census_name': 'B03002_003E'},
                      {'desc_name': 'Hispanic', 'census_name': 'B03002_012E'},
                      {'desc_name': 'TotBlk', 'census_name': 'B02001_003E'},
                      {'desc_name': 'TotAI', 'census_name': 'B02001_004E'},
                      {'desc_name': 'TotAsian', 'census_name': 'B02001_005E'},
                      {'desc_name': 'Tot_HPI', 'census_name': 'B02001_006E'},
                      {'desc_name': 'TotOther', 'census_name': 'B02001_007E'},
                      {'desc_name': 'Tot2Race', 'census_name': 'B02001_008E'},
                      {'desc_name': 'TotPSK', 'census_name': 'C17002_001E'},
                      {'desc_name': 'BlwPov_Under50', 'census_name': 'C17002_002E'},
                      {'desc_name': 'BlwPov_50to99', 'census_name': 'C17002_003E'},
                      {'desc_name': 'BlwPov_100to124', 'census_name': 'C17002_004E'},
                      {'desc_name': 'PopOver5', 'census_name': 'B16004_001E'},
                      {'desc_name': 'SpeakSpanish_5_17', 'census_name': 'B16004_004E'},
                      {'desc_name': 'SpeakSpanish_5_17_EnglishVWell', 'census_name': 'B16004_005E'},
                      {'desc_name': 'SpeakIE_5_17', 'census_name': 'B16004_009E'},
                      {'desc_name': 'SpeakIE_5_17_EnglishVWell', 'census_name': 'B16004_010E'},
                      {'desc_name': 'SpeakAsian_5_17', 'census_name': 'B16004_014E'},
                      {'desc_name': 'SpeakAsian_5_17_EnglishVWell', 'census_name': 'B16004_015E'},
                      {'desc_name': 'SpeakOther_5_17', 'census_name': 'B16004_019E'},
                      {'desc_name': 'SpeakOther_5_17_EnglishVWell', 'census_name': 'B16004_020E'},
                      {'desc_name': 'SpeakSpanish_18_64', 'census_name': 'B16004_026E'},
                      {'desc_name': 'SpeakSpanish_18_64_EnglishVWell', 'census_name': 'B16004_027E'},
                      {'desc_name': 'SpeakIE_18_64', 'census_name': 'B16004_031E'},
                      {'desc_name': 'SpeakIE_18_64_EnglishVWell', 'census_name': 'B16004_032E'},
                      {'desc_name': 'SpeakAsian_18_64', 'census_name': 'B16004_036E'},
                      {'desc_name': 'SpeakAsian_18_64_EnglishVWell', 'census_name': 'B16004_037E'},
                      {'desc_name': 'SpeakOther_18_64', 'census_name': 'B16004_041E'},
                      {'desc_name': 'SpeakOther_18_64_EnglishVWell', 'census_name': 'B16004_042E'},
                      {'desc_name': 'SpeakSpanish_65Over', 'census_name': 'B16004_048E'},
                      {'desc_name': 'SpeakSpanish_65Over_EnglishVWell', 'census_name': 'B16004_049E'},
                      {'desc_name': 'SpeakIE_65Over', 'census_name': 'B16004_053E'},
                      {'desc_name': 'SpeakIE_65Over_EnglishVWell', 'census_name': 'B16004_054E'},
                      {'desc_name': 'SpeakAsian_65Over', 'census_name': 'B16004_058E'},
                      {'desc_name': 'SpeakAsian_65Over_EnglishVWell', 'census_name': 'B16004_059E'},
                      {'desc_name': 'SpeakOther_65Over', 'census_name': 'B16004_063E'},
                      {'desc_name': 'SpeakOther_65Over_EnglishVWell', 'census_name': 'B16004_064E'},
                     ]
bg_desired_col2 =    [{'desc_name': 'Age14Under1', 'census_name': 'B01001_003E'},
                      {'desc_name': 'Age14Under2', 'census_name': 'B01001_004E'},
                      {'desc_name': 'Age14Under3', 'census_name': 'B01001_005E'},
                      {'desc_name': 'Age14Under4', 'census_name': 'B01001_027E'},
                      {'desc_name': 'Age14Under5', 'census_name': 'B01001_028E'},
                      {'desc_name': 'Age14Under6', 'census_name': 'B01001_029E'},
                      {'desc_name': 'Pop18Over', 'census_name': 'B21001_001E'},
                      {'desc_name': 'TotalVet', 'census_name': 'B21001_002E'},
                      {'desc_name': 'Age65Over1', 'census_name': 'B01001_020E'},
                      {'desc_name': 'Age65Over2', 'census_name': 'B01001_021E'},
                      {'desc_name': 'Age65Over3', 'census_name': 'B01001_022E'},
                      {'desc_name': 'Age65Over4', 'census_name': 'B01001_023E'},
                      {'desc_name': 'Age65Over5', 'census_name': 'B01001_024E'},
                      {'desc_name': 'Age65Over6', 'census_name': 'B01001_025E'},
                      {'desc_name': 'Age65Over7', 'census_name': 'B01001_044E'},
                      {'desc_name': 'Age65Over8', 'census_name': 'B01001_045E'},
                      {'desc_name': 'Age65Over9', 'census_name': 'B01001_046E'},
                      {'desc_name': 'Age65Over10', 'census_name': 'B01001_047E'},
                      {'desc_name': 'Age65Over11', 'census_name': 'B01001_048E'},
                      {'desc_name': 'Age65Over12', 'census_name': 'B01001_049E'},
                      {'desc_name': 'TotalHH', 'census_name': 'B11005_001E'},
                      {'desc_name': 'FHH_Family', 'census_name': 'B11005_007E'},
                      {'desc_name': 'FHH_NonFamily', 'census_name': 'B11005_010E'},
                      {'desc_name': 'ZCHH_Owner', 'census_name': 'B25044_003E'},
                      {'desc_name': 'ZCHH_Renter', 'census_name': 'B25044_010E'}
                     ]

# List of dictionaries describing the data you're requesting from the Census API at the TRACT geography.
# desc_name is a descriptive name of your choosing. This becomes part of the field name. Choose something tidy
# that will result in a recognizable field name later.
# census_name is the variable name as described here: https://api.census.gov/data/2019/acs/acs5/variables.html

tract_desired_columns = [{'desc_name': 'TotPopTract', 'census_name': 'B18101_001E'},
                         {'desc_name': 'MaleDisabUnder5', 'census_name': 'B18101_004E'},
                         {'desc_name': 'MaleDisab5to17', 'census_name': 'B18101_007E'},
                         {'desc_name': 'MaleDisab18to34', 'census_name': 'B18101_010E'},
                         {'desc_name': 'MaleDisab35to64', 'census_name': 'B18101_013E'},
                         {'desc_name': 'MaleDisab65to74', 'census_name': 'B18101_016E'},
                         {'desc_name': 'MaleDisab75over', 'census_name': 'B18101_019E'},
                         {'desc_name': 'FemDisabUnder5', 'census_name': 'B18101_023E'},
                         {'desc_name': 'FemDisab5to17', 'census_name': 'B18101_026E'},
                         {'desc_name': 'FemDisab18to34', 'census_name': 'B18101_029E'},
                         {'desc_name': 'FemDisab35to64', 'census_name': 'B18101_032E'},
                         {'desc_name': 'FemDisab65to74', 'census_name': 'B18101_035E'},
                         {'desc_name': 'FemDisab75over', 'census_name': 'B18101_038E'}
                        ]

# List of county FIPS codes the script will request from the API. Also clips the input
# block group and tract geographies.
state = '17'
counties = {
    '031':'Cook',
    '043':'DuPage',
    '089':'Kane',
    '093':'Kendall',
    '097':'Lake',
    '111':'McHenry',
    '197':'Will'
}
year = '2022'


In [34]:
output_folder = ''
parent_folder = ''

## TRACT API CALL ##

# Initializes list of column names to be inserted into URL for TRACT API call.
df_init_columns = []
census_column_names = ''

for x in tract_desired_columns:
    df_init_columns.append(f"{x['desc_name']}")
    census_column_names += x['census_name'] + ','

df_init_columns.extend(['State','County','Tract'])
census_column_names = census_column_names[:-1]

# Establishes empty pandas dataframe to load data from API into.
print("Creating initial empty dataframe for tracts...")
results_pd_all_tract = pd.DataFrame(columns = df_init_columns)

Creating initial empty dataframe for tracts...


In [35]:
url = f'https://api.census.gov/data/{year}/acs/acs5?get={census_column_names}&in=state:{state}%20county:{county}&for=tract'
print(url)

https://api.census.gov/data/2022/acs/acs5?get=B18101_001E,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E&in=state:17%20county:031&for=tract


In [36]:
# Iterates through each county in counties list to connect to the URL, download the results in JSON format, and insert them into the empty data frame.
for county in counties:
    print("Processing county {}...".format(county))

    url = f'https://api.census.gov/data/{year}/acs/acs5?get={census_column_names}&in=state:{state}%20county:{county}&for=tract'

    # print("Requesting URL...")
    apicall = call.request('GET', url)

    # print("Loading results text...")
    results = apicall.data

    # print("Parsing into json...")
    results_json = json.loads(results)[1:]

    # print("Loading JSON into Pandas dataframe...")
    results_pd = pd.DataFrame(columns = df_init_columns, data = results_json)

    results_pd_all_tract = pd.concat([results_pd_all_tract, results_pd])

#Establish a new column in the dataframe with a concatenated GEOID from State, County, and Tract FIPS codes.
# print("Calculating tract geoid for all counties...")
cols = ['State', 'County', 'Tract']
for c in cols:
    results_pd_all_tract[c] = results_pd_all_tract[c].astype(str)
results_pd_all_tract['Tract_GEOID'] = results_pd_all_tract['State'] + results_pd_all_tract['County'] + results_pd_all_tract['Tract']

## TRACT CALCULATIONS AND REFORMATTING ##

print('Usings Pandas to calculate and reorder fields (Tracts)...')

for col in list(results_pd_all_tract.columns):
    if col not in ['Tract_GEOID','County']:
        results_pd_all_tract.loc[~results_pd_all_tract[col].apply(lambda x: str(x).isnumeric()), col] = 0
        results_pd_all_tract[col] = results_pd_all_tract[col].astype('int64')

results_pd_all_tract['WholeTract_PWD'] = 0
for gender in ['Male','Fem']:
    for age in ['Under5','5to17','18to34','35to64','65to74','75over']:
        results_pd_all_tract['WholeTract_PWD'] = results_pd_all_tract['WholeTract_PWD'] + results_pd_all_tract[f'{gender}Disab{age}']

Processing county 031...
Processing county 043...
Processing county 089...
Processing county 093...
Processing county 097...
Processing county 111...
Processing county 197...
Usings Pandas to calculate and reorder fields (Tracts)...


In [37]:
## BLOCK GROUP API CALL 1 ##
   
# Initializes list of column names to be inserted into URL for BLOCK GROUP API call.
df_init_columns = []
census_column_names = ''

for x in bg_desired_col1:
    df_init_columns.append(f"{x['desc_name']}")
    census_column_names = census_column_names + x['census_name'] + ','

df_init_columns.extend(['State','County','Tract','BG'])
census_column_names = census_column_names[:-1]

# Establishes empty pandas dataframe to load data from BLOCK GROUP API call into.
print("Creating initial empty dataframe for block groups...")
results_pd_all_bg1 = pd.DataFrame(columns = df_init_columns)

# Iterates through each county in counties list to connect to the URL, download the results in JSON format, and insert them into the empty data frame.
for county in counties:
    print(f"Processing county {county}...")

    url = f'https://api.census.gov/data/{year}/acs/acs5?get={census_column_names}&in=state:{state}%20county:{county}&for=block%20group'

    # print("Requesting URL...")    
    apicall = call.request('GET', url)

    # print("Loading results text...")
    results = apicall.data

    # print("Parsing into json...")
    results_json = json.loads(results)[1:]

    # print("Loading JSON into Pandas dataframe...")
    results_pd = pd.DataFrame(columns = df_init_columns, data = results_json)

    results_pd_all_bg1 = pd.concat([results_pd_all_bg1, results_pd])

#Establish a new column in the dataframe with a concatenated GEOID from State, County, Tract, and BG FIPS codes.
# print("Calculating tract geoid for all counties...")
results_pd_all_bg1['GEOID'] = results_pd_all_bg1['State'] + results_pd_all_bg1['County'] + results_pd_all_bg1['Tract'] + results_pd_all_bg1['BG']

## BLOCK GROUP API CALL 2 ##
   
# Initializes list of column names to be inserted into URL for BLOCK GROUP API call.
df_init_columns = []
census_column_names = ''

for x in bg_desired_col2:
    df_init_columns.append(f"{x['desc_name']}")
    census_column_names = census_column_names + x['census_name'] + ','

df_init_columns.extend(['State','County','Tract','BG'])
census_column_names = census_column_names[:-1]

# Establishes empty pandas dataframe to load data from BLOCK GROUP API call into.
print("Creating initial empty dataframe for block groups...")
results_pd_all_bg2 = pd.DataFrame(columns = df_init_columns)

# Iterates through each county in counties list to connect to the URL, download the results in JSON format, and insert them into the empty data frame.
for county in counties:
    print(f"Processing county {county}...")

    url = f'https://api.census.gov/data/{year}/acs/acs5?get={census_column_names}&in=state:{state}%20county:{county}&for=block%20group'

    # print("Requesting URL...")
    apicall = call.request('GET', url)

    # print("Loading results text...")
    results = apicall.data

    # print("Parsing into json...")
    results_json = json.loads(results)[1:]

    # print("Loading JSON into Pandas dataframe...")
    results_pd = pd.DataFrame(columns = df_init_columns, data = results_json)

    results_pd_all_bg2 = pd.concat([results_pd_all_bg2, results_pd])

#Establish a new column in the dataframe with a concatenated GEOID from State, County, Tract, and BG FIPS codes.
# print("Calculating tract geoid for all counties...")
results_pd_all_bg2['GEOID'] = results_pd_all_bg2['State'] + results_pd_all_bg2['County'] + results_pd_all_bg2['Tract'] + results_pd_all_bg2['BG']
results_pd_all_bg2['Tract_GEOID'] = results_pd_all_bg2['State'] + results_pd_all_bg2['County'] + results_pd_all_bg2['Tract']


#Join the two BG-level data frames together.
results_pd_notract_bg = pd.merge(results_pd_all_bg1,results_pd_all_bg2,on='GEOID')

#Join tract-level PWD data to block groups.
results_pd_all_bg = pd.merge(results_pd_notract_bg,results_pd_all_tract,on='Tract_GEOID',how='left',suffixes=('_x','_y'))

Creating initial empty dataframe for block groups...
Processing county 031...
Processing county 043...
Processing county 089...
Processing county 093...
Processing county 097...
Processing county 111...
Processing county 197...
Creating initial empty dataframe for block groups...
Processing county 031...
Processing county 043...
Processing county 089...
Processing county 093...
Processing county 097...
Processing county 111...
Processing county 197...


In [38]:
## BLOCK GROUP CALCULATIONS AND REFORMATTING ##

print('Usings Pandas to calculate and reorder fields (Block Groups)...')

for col in list(results_pd_all_bg.columns):
    if col not in ['GEOID','County_x']:
        results_pd_all_bg.loc[~results_pd_all_bg[col].apply(lambda x: str(x).isnumeric()), col] = 0
        results_pd_all_bg[col] = results_pd_all_bg[col].astype('int64')

results_pd_all_bg['TotalMin'] = results_pd_all_bg['Total_Pop'] - results_pd_all_bg ['NotHispLatino_WhiteAlone']

results_pd_all_bg['BlwPov'] = results_pd_all_bg['BlwPov_Under50'] + results_pd_all_bg['BlwPov_50to99'] + results_pd_all_bg['BlwPov_100to124']

results_pd_all_bg['TotalLEP'] = 0
for language in ['Spanish','IE','Asian','Other']:
    results_pd_all_bg['{}LEP'.format(language)] = 0
    for age in ['5_17','18_64','65Over']:
        results_pd_all_bg['{}LEP'.format(language)] += (results_pd_all_bg['Speak{}_{}'.format(language,age)] - results_pd_all_bg['Speak{}_{}_EnglishVWell'.format(language,age)])
    results_pd_all_bg['TotalLEP'] += results_pd_all_bg['{}LEP'.format(language)]

results_pd_all_bg['Age65Over'] = 0
for i in range(1,12+1):
    results_pd_all_bg['Age65Over'] += results_pd_all_bg['Age65Over{}'.format(str(i))]

results_pd_all_bg['Age14Under'] = 0
for i in range(1,6+1):
    results_pd_all_bg['Age14Under'] += results_pd_all_bg['Age14Under{}'.format(str(i))]

results_pd_all_bg['TotalFHH'] = results_pd_all_bg['FHH_Family'] + results_pd_all_bg['FHH_NonFamily']

results_pd_all_bg['NoCar'] = results_pd_all_bg['ZCHH_Owner'] + results_pd_all_bg['ZCHH_Renter']

results_pd_all_bg['Sum_PWD'] = results_pd_all_bg['WholeTract_PWD'].astype('float64') * (results_pd_all_bg['Total_Pop'].astype('float64') / results_pd_all_bg['TotPopTract'].astype('float64'))


calculation_fields = [{'variable': 'TotalMin', 'universe': 'Total_Pop', 'pct': 'Pct_TotMin', 'ratio': 'Rat_TotMin'},
                      {'variable': 'Hispanic', 'universe': 'Total_Pop', 'pct': 'Pct_Hisp', 'ratio': 'Rat_Hisp'},
                      {'variable': 'TotBlk', 'universe': 'Total_Pop', 'pct': 'Pct_TotBlk', 'ratio': 'Rat_TotBlk'},
                      {'variable': 'TotAI', 'universe': 'Total_Pop', 'pct': 'Pct_TotAI', 'ratio': 'Rat_TotAI'},
                      {'variable': 'TotAsian', 'universe': 'Total_Pop', 'pct': 'Pct_TotAsn', 'ratio': 'Rat_TotAsn'},
                      {'variable': 'Tot_HPI', 'universe': 'Total_Pop', 'pct': 'Pct_TotHPI', 'ratio': 'Rat_TotHPI'},
                      {'variable': 'TotOther', 'universe': 'Total_Pop', 'pct': 'Pct_TotOth', 'ratio': 'Rat_TotOth'},
                      {'variable': 'Tot2Race', 'universe': 'Total_Pop', 'pct': 'Pct_Tot2Ra', 'ratio': 'Rat_Tot2Ra'},
                      {'variable': 'BlwPov', 'universe': 'TotPSK', 'pct': 'Pct_BlwPov', 'ratio': 'Rat_BlwPov'},
                      {'variable': 'TotalLEP', 'universe': 'PopOver5', 'pct': 'Pct_TotLEP', 'ratio': 'Rat_TotLEP'},
                      {'variable': 'SpanishLEP', 'universe': 'PopOver5', 'pct': 'Pct_SpLEP', 'ratio': 'Rat_SpLEP'},
                      {'variable': 'IELEP', 'universe': 'PopOver5', 'pct': 'Pct_IE_LEP', 'ratio': 'Rat_IE_LEP'},
                      {'variable': 'AsianLEP', 'universe': 'PopOver5', 'pct': 'Pct_AsnLEP', 'ratio': 'Rat_AsnLEP'},
                      {'variable': 'OtherLEP', 'universe': 'PopOver5', 'pct': 'Pct_OthLEP', 'ratio': 'Rat_OthLEP'},
                      {'variable': 'Age65Over', 'universe': 'Total_Pop', 'pct': 'Pct65_Over', 'ratio': 'Rat_65Over'},
                      {'variable': 'TotalFHH', 'universe': 'TotalHH', 'pct': 'Pct_TotFHH', 'ratio': 'Rat_TotFHH'},
                      {'variable': 'NoCar', 'universe': 'TotalHH', 'pct': 'Pct_NoCar', 'ratio': 'Rat_NoCar'},
                      {'variable': 'Age14Under', 'universe': 'Total_Pop', 'pct': 'Pct14_Unde', 'ratio': 'Rat_14Unde'},
                      {'variable': 'TotalVet', 'universe': 'Pop18Over', 'pct': 'Pct_Vet', 'ratio': 'Rat_Vet'},
                      {'variable': 'Sum_PWD', 'universe': 'Total_Pop', 'pct': 'Pct_PWD', 'ratio':'Rat_PWD'}
                     ]                      

for field in calculation_fields:
    results_pd_all_bg[field['pct']] = (results_pd_all_bg[field['variable']] / results_pd_all_bg[field['universe']])
#    regional_pct = ((np.asarray(results_pd_all_bg.iloc[:,results_pd_all_bg.columns.get_loc(field['variable'])], dtype=np.float64).sum()) / (np.asarray(results_pd_all_bg.iloc[:,results_pd_all_bg.columns.get_loc(field['universe'])], dtype=np.float64).sum()))
    regional_pct = float(results_pd_all_bg[field['variable']].sum()) / float(results_pd_all_bg[field['universe']].sum())
    results_pd_all_bg[field['ratio']] = results_pd_all_bg[field['pct']] / regional_pct

# results_pd_all_bg.loc[results_pd_all_bg['Rat_65Over'] >= 1.0, 'ARP_65Over'] = 'Y'
# results_pd_all_bg.loc[results_pd_all_bg['Rat_65Over'] < 1.0, 'ARP_65Over'] = 'N'

# results_pd_all_bg.loc[results_pd_all_bg['Rat_BlwPov'] >= 1.0, 'ARP_BlwPov'] = 'Y'
# results_pd_all_bg.loc[results_pd_all_bg['Rat_BlwPov'] < 1.0, 'ARP_BlwPov'] = 'N'

# results_pd_all_bg.loc[results_pd_all_bg['Rat_PWD'] >= 1.0, 'ARP_PWD'] = 'Y'
# results_pd_all_bg.loc[results_pd_all_bg['Rat_PWD'] < 1.0, 'ARP_PWD'] = 'N'

results_pd_all_bg['CountyText'] = 'temp'
for county in counties:
    results_pd_all_bg.loc[results_pd_all_bg['County_x'].astype(str) == county, 'CountyText'] = counties[county]

#results_bg_reordered = results_pd_all_bg[['GEOID','CountyText','Total_Pop','TotalMin','Pct_TotMin','Rat_TotMin','Hispanic','Pct_Hisp','Rat_Hisp','TotBlk','Pct_TotBlk','Rat_TotBlk','TotAI','Pct_TotAI','Rat_TotAI','TotAsian','Pct_TotAsn','Rat_TotAsn','Tot_HPI','Pct_TotHPI','Rat_TotHPI','TotOther','Pct_TotOth','Rat_TotOth','Tot2Race','Pct_Tot2Ra','Rat_Tot2Ra','TotPSK','BlwPov','Pct_BlwPov','Rat_BlwPov','PopOver5','TotalLEP','Pct_TotLEP','Rat_TotLEP','SpanishLEP','Pct_SpLEP','Rat_SpLEP','IELEP','Pct_IE_LEP','Rat_IE_LEP','AsianLEP','Pct_AsnLEP','Rat_AsnLEP','OtherLEP','Pct_OthLEP','Rat_OthLEP','Age65Over','Pct65_Over','Rat_65Over','TotalHH','TotalFHH','Pct_TotFHH','Rat_TotFHH','NoCar','Pct_NoCar','Rat_NoCar','Min_RegPct','Pov_RegPct','Both_RegPct']]
results_bg_reordered = results_pd_all_bg[['GEOID','Tract_GEOID','CountyText','Total_Pop','TotalMin','Pct_TotMin','Hispanic','Pct_Hisp','TotBlk','Pct_TotBlk','TotAI','Pct_TotAI','TotAsian','Pct_TotAsn','Tot_HPI','Pct_TotHPI','TotOther','Pct_TotOth','Tot2Race','Pct_Tot2Ra','TotPSK','BlwPov','Pct_BlwPov','Rat_BlwPov','PopOver5','TotalLEP','Pct_TotLEP','SpanishLEP','Pct_SpLEP','IELEP','Pct_IE_LEP','AsianLEP','Pct_AsnLEP','OtherLEP','Pct_OthLEP','Age65Over','Pct65_Over','Rat_65Over','TotalHH','NoCar','Pct_NoCar','Rat_NoCar','Age14Under','Pct14_Unde','Rat_14Unde','Pop18Over','TotalVet','Pct_Vet','Rat_Vet','TotPopTract','Sum_PWD','Pct_PWD','Rat_PWD']]

results_bg_reordered = results_bg_reordered.rename(columns={'CountyText':'County'})
results_bg_reordered = results_bg_reordered.rename(columns={'Tract_GEOID':'TractID'})

Usings Pandas to calculate and reorder fields (Block Groups)...


/data/user/0/ru.iiec.pydroid3/cache/ipykernel_10265/506805413.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_pd_all_bg['NoCar'] = results_pd_all_bg['ZCHH_Owner'] + results_pd_all_bg['ZCHH_Renter']
/data/user/0/ru.iiec.pydroid3/cache/ipykernel_10265/506805413.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_pd_all_bg['Sum_PWD'] = results_pd_all_bg['WholeTract_PWD'].astype('float64') * (results_pd_all_bg['Total_Pop'].astype('float64') / results_pd_all_bg['TotPopTract'].astype('float64'))
/data/user/0/ru.i

In [42]:
bg = results_bg_reordered.copy()
bg.loc[bg['County']=='Lake']

,GEOID,TractID,County,Total_Pop,TotalMin,Pct_TotMin,Hispanic,Pct_Hisp,TotBlk,Pct_TotBlk,...,Pct14_Unde,Rat_14Unde,Pop18Over,TotalVet,Pct_Vet,Rat_Vet,TotPopTract,Sum_PWD,Pct_PWD,Rat_PWD
5013,170978601031,17097860103,Lake,696,150,0.215517,43,0.061782,56,0.080460,...,0.012931,0.070834,668,40,0.059880,1.505310,3677,79.121023,0.113680,1.123285
5014,170978601032,17097860103,Lake,848,208,0.245283,163,0.192217,9,0.010613,...,0.143868,0.788087,703,32,0.045519,1.144293,3677,96.400326,0.113680,1.123285
5015,170978601033,17097860103,Lake,2148,461,0.214618,344,0.160149,83,0.038641,...,0.117318,0.642653,1803,237,0.131448,3.304418,3677,244.183846,0.113680,1.123285
5016,170978601041,17097860104,Lake,1906,251,0.131689,211,0.110703,8,0.004197,...,0.183106,1.003027,1510,128,0.084768,2.130961,2940,311.831973,0.163605,1.616610
5017,170978601042,17097860104,Lake,1048,111,0.105916,22,0.020992,14,0.013359,...,0.088740,0.486107,955,168,0.175916,4.422301,2940,171.458503,0.163605,1.616610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5433,170978661002,17097866100,Lake,1776,1703,0.958896,1492,0.840090,299,0.168356,...,0.293356,1.606959,1147,59,0.051439,1.293097,3570,160.188235,0.090196,0.891241
5434,170978662001,17097866200,Lake,2725,279,0.102385,25,0.009174,33,0.012110,...,0.100917,0.552811,2286,137,0.059930,1.506561,6113,356.617046,0.130869,1.293133
5435,170978662002,17097866200,Lake,1868,339,0.181478,81,0.043362,0,0.000000,...,0.145075,0.794699,1597,52,0.032561,0.818542,6113,244.462621,0.130869,1.293133
5436,170978662003,17097866200,Lake,1663,245,0.147324,75,0.045099,52,0.031269,...,0.131088,0.718083,1389,93,0.066955,1.683151,6113,217.634549,0.130869,1.293133
